# Tracking Data
   - Data about in-game actions and trajectories. 
   <br>
   <br>
   - Most tracking data is collected by utilizing machine vision to gather precise spatio-temporal information.
   <br>
   <br>
   - Can be independent from the sport itself.


### Events and Trajectories
   - Often adopting a third-person perspective on the data
   <br>
   <br>
   - **Types of Visualization:**
       - Event density map.
       - Events representation.
       - Trajectories.
       - Detailed events with context.
       - Real-time.
       
       <div style="text-align:center"><img width="1200px" src="./images/basketball_shot_distribution_density.png"/></div>
       <br>
       <br>
       <div style="text-align:center">Figure 1: Shot distribution and density in CourtVision (Goldsberry K., 2012).</div>

### Players’ point of view
   - Often adopting a first-person perspective on the data
   <br>
   <br>
   - **Types of Visualization:**
       - First-person.
       - Embedded tracking and visualization.
       
       
<div style="text-align:center"><img width="500px" src="./images/penalty_shooter_point_of_view.png"/></div>
<br>
<br>
<div style="text-align:center">Figure 2: A penalty shooter’s point of view is reconstructed in 3D, showing her favorite target areas around the goalkeeper (Rusu A., Stoica D., Burns E.).</div>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("notebook")
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
pd.options.mode.chained_assignment = None

# Detailed NFL Play-by-Play Data (2009-2017)

## Load Data

Dataset: https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016

Source: https://www.kaggle.com/carrie1/nfl-te-qb-duo-performance

In [ ]:
df = pd.read_csv('../datasets/nflplaybyplay2009to2016/NFL Play by Play 2009-2017 (v4).csv')
df.head(5)

### Data cleaning

In [ ]:
columns = ["Date", "posteam", "desc", "PassAttempt", "ydsnet", "sp"]
df = df[columns]
df.head(10)

In [ ]:
# Find Dallas plays with Witten and Romo in them
witten_romo = df[(df["posteam"] == 'DAL')  & (df["desc"].str.contains("J.Witten")) &
                 (df["desc"].str.contains("T.Romo")) & (df['PassAttempt'] == 1)]
# Find New England plays with Brady and the Gronk
gronk_brady = df[(df["posteam"] == 'NE') & (df["desc"].str.contains("T.Brady")) &
                 (df["desc"].str.contains("R.Gronkowski")) & (df['PassAttempt'] == 1)]
# Find Titans plays with Walker and Mariota
walker_mariota = df[(df["posteam"] == 'TEN') & (df["desc"].str.contains("D.Walker")) &
                    (df["desc"].str.contains("M.Mariota")) & (df['PassAttempt'] == 1)]
# Find New England plays with Brady and the Gronk
gronk_brady = df[(df["posteam"] == 'NE') & (df["desc"].str.contains("T.Brady")) &
                 (df["desc"].str.contains("R.Gronkowski")) & (df['PassAttempt'] == 1)]
# Find Bengals plays with Eifert & Dalton
eifert_dalton = df[(df["posteam"] == 'CIN') & (df["desc"].str.contains("T.Eifert")) & 
               (df["desc"].str.contains("A.Dalton")) & (df['PassAttempt'] == 1)]
# Find Carolina plays with Eifert & Dalton
olsen_cam = df[(df["posteam"] == 'CAR') & (df["desc"].str.contains("G.Olsen")) & 
               (df["desc"].str.contains("C.Newton")) & (df['PassAttempt'] == 1)]
# Find Chargers plays with Gates and Rivers
gates_rivers = df[(df["posteam"] == 'SD') & (df["desc"].str.contains("A.Gates")) & 
               (df["desc"].str.contains("P.Rivers")) & (df['PassAttempt'] == 1)]
# Find Saints plays with Graham and Brees
graham_brees = df[(df["posteam"] == 'NO') & (df["desc"].str.contains("J.Graham")) & 
               (df["desc"].str.contains("D.Brees")) & (df['PassAttempt'] == 1)]
# Find Seattle plays with Graham and Wilson
graham_wilson = df[(df["posteam"] == 'SEA') & (df["desc"].str.contains("J.Graham")) & 
               (df["desc"].str.contains("R.Wilson")) & (df['PassAttempt'] == 1)]
# Find Philadelphia plays with Bradford and Ertz
bradford_ertz = df[(df["posteam"] == 'PHI') & (df["desc"].str.contains("S.Bradford")) & 
               (df["desc"].str.contains("Z.Ertz")) & (df['PassAttempt'] == 1)]
# Find Detroit plays with Stafford and Ebron
ebron_staff = df[(df["posteam"] == 'DET') & (df["desc"].str.contains("M.Stafford")) & 
               (df["desc"].str.contains("E.Ebron")) & (df['PassAttempt'] == 1)]
# Find Jacksonville plays with Bortles and Thomas
thomas_bortles = df[(df["posteam"] == 'JAC') & (df["desc"].str.contains("B.Bortles")) & 
               (df["desc"].str.contains("J.Thomas")) & (df['PassAttempt'] == 1)]
# Find Kansas City plays with Smith and Kelce
smith_kelce = df[(df["posteam"] == 'KC') & (df["desc"].str.contains("A.Smith")) & 
               (df["desc"].str.contains("T.Kelce")) & (df['PassAttempt'] == 1)]

In [ ]:
witten_romo['duo'] = 'Witten & Romo'
gronk_brady['duo'] = 'Gronkowski & Brady'
olsen_cam['duo'] = 'Olsen & Newton'
eifert_dalton['duo'] = 'Eifter & Dalton'
walker_mariota['duo'] = 'Walker & Mariota'
gates_rivers['duo'] ='Gates & Rivers'
graham_brees['duo'] ='Graham & Brees'
graham_wilson['duo'] ='Graham & Wilson'
bradford_ertz['duo'] = 'Ertz & Bradford'
ebron_staff['duo']= 'Ebron & Stafford'
thomas_bortles['duo'] = 'Thomas & Bortles'
smith_kelce['duo']= 'Kelce & Smith'


frames = [gronk_brady, witten_romo, olsen_cam, eifert_dalton, walker_mariota, gates_rivers,
          graham_brees,graham_wilson,bradford_ertz,ebron_staff,thomas_bortles,smith_kelce]
df2 = pd.concat(frames)
df2.head(10)

### Data statistics

In [ ]:
df2["duo"].value_counts()

In [ ]:
df2['Season'] = df2['Date'].str[:4]
ct = pd.crosstab(df2['duo'], df2['Season'])
plt.figure(figsize=(10, 8))
ax = sns.heatmap(ct, annot=True, fmt="d").set_title('Number of Pass Plays Per Season')

## Net Yards per Play

**Net Yards:** This statistic is calculated by subtracting a team's starting position on the field from the team's ending field position

**Swarm Plot:** Draw a categorical scatterplot with non-overlapping points. 

In [ ]:
plt.figure(figsize=(10, 25))
ax = sns.swarmplot(x="ydsnet", y="duo", hue="sp", data=df2, size=4)
leg = plt.legend(bbox_to_anchor=(1.05, 1), loc=2)

Done!